#### KPI 4: CO2 emissions (top down)
---
Combined data frame for every entity in use case with:
- grey_energy [kWh] -> to be multiplied by co2 emission factor
- grey_energy [%] -> share grey energy consumed (1 - share renewable)

In [2]:
import os
import pandas as pd

from functions_topdown_v2 import *

home_path_new = r'C:\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\07_Arbeitsordner\04_Ergebnisse\20220920_Ergebnisse_4_Wochen\new_results_2023'
results_new = r'C:\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\07_Arbeitsordner\04_Ergebnisse\20220920_Ergebnisse_4_Wochen\new_results_2023\results'

home_path_old = r'C:\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\07_Arbeitsordner\04_Ergebnisse\20220920_Ergebnisse_4_Wochen'
results_old = r'C:\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\07_Arbeitsordner\04_Ergebnisse\20220920_Ergebnisse_4_Wochen\results'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### Instructions how to use function ```co2_calcuation```:
- enter the relevant simulation data directory for old results (```home_path_old```) or new results (```home_path_new```)
- enter the relevant use case number as an integer
- save the result in a pandas dataframe
- save the dataframe as a xlsx-file to old results folder (path: ```results_old```) or new results folder (path: ```results_new```)

In [ ]:
# calculate co2 emissions for old data, use case 4
df4 = co2_calculation(home_path_old, use_case_nr=4)

In [ ]:
# save dataframe to xlsx-file in old results path
df4.to_excel(os.path.join(results_old, 'use_case_4_topdown.xlsx'))